In [2]:
import os
import numpy as np 
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate , LeakyReLU, BatchNormalization , ReLU
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

# from tensorflow.keras.engine.topology import Layer
import keras.backend as K
# from keras_frcnn import vgg16 as nn
import pandas as pd
import numpy as np
import ast
import cv2
# import imutils
import matplotlib.pyplot as plt
from PIL import Image

In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

Với kích thước đầu vào là một list của 2 tensor 4D [X_img, X_roi] với kích thước :
X_img:
(1, rows, cols, channels)
X_roi:
(1,num_rois,4) list of rois, with ordering (x,y,w,h)
Và kích thước đầu ra: là một tensor 3D:
(1, num_rois, channels, pool_size, pool_size)

In [12]:
class RoiPoolingConv(keras.layers.Layer):
    '''
    # Arguments
        pool_size: int
            Size of pooling region to use. pool_size = 7 will result in a 7x7 region.
        num_rois: number of regions of interest to be used
    '''
    def __init__(self, pool_size, num_rois, **kwargs):

        self.dim_ordering = K.common.image_dim_ordering()
        self.pool_size = pool_size
        self.num_rois = num_rois
        self.nb_channels = 

        super(RoiPoolingConv, self).__init__(**kwargs)

    # def build(self, input_shape):
    #     self.nb_channels = input_shape[0][3]   

    def compute_output_shape(self, input_shape):
        return None, self.num_rois, self.pool_size, self.pool_size, self.nb_channels

    def call(self, x, mask=None):

        assert(len(x) == 2)

        # x[0] is image with shape (rows, cols, channels)
        img = x[0]

        # x[1] is roi with shape (num_rois,4) with ordering (x,y,w,h)
        rois = x[1]

        input_shape = K.shape(img)

        outputs = []

        for roi_idx in range(self.num_rois):

            x = rois[0, roi_idx, 0]
            y = rois[0, roi_idx, 1]
            w = rois[0, roi_idx, 2]
            h = rois[0, roi_idx, 3]

            x = K.cast(x, 'int32')
            y = K.cast(y, 'int32')
            w = K.cast(w, 'int32')
            h = K.cast(h, 'int32')

            # Resized roi of the image to pooling size (7x7)
            rs = tf.image.resize(img[:, y:y+h, x:x+w, :], (self.pool_size, self.pool_size))
            outputs.append(rs)
                

        final_output = K.concatenate(outputs, axis=0)

        # Reshape to (1, num_rois, pool_size, pool_size, nb_channels)
        # Might be (1, 4, 7, 7, 3)
        final_output = K.reshape(final_output, (1, self.num_rois, self.pool_size, self.pool_size, self.nb_channels))

        # permute_dimensions is similar to transpose
        final_output = K.permute_dimensions(final_output, (0, 1, 2, 3, 4))

        return final_output
    
    
    def get_config(self):
        config = {'pool_size': self.pool_size,
                  'num_rois': self.num_rois}
        base_config = super(RoiPoolingConv, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [4]:
input_shape_img = (None, None, 3)

In [6]:
img_input = Input(shape=input_shape_img)
roi_input = Input(shape=(None, 4))

shared_layers = VGG16(weights='imagenet', include_top=False, input_shape=input_shape_img)

In [14]:
shared_layers

In [10]:
a  = RoiPoolingConv(7, 10000)([shared_layers, roi_input])

TypeError: 'NoneType' object is not subscriptable